In [1]:
import pystac_client
import planetary_computer as pc

#import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import shapely
import os
from shapely.geometry import box


In [2]:
def query_geom(geom, year):

    date_range = str(year)+'-01-01/'+str(year)+'-12-31'

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1")

    search = catalog.search(
        collections=["naip"], 
        intersects=geom, 
        datetime=date_range)
    
    items =list(search.get_items()) 
    if len(items)==0:
        return None
    return items

In [3]:
#fp = os.path.join(os.getcwd(),'sb_county_coast','sb_county_coast.shp')
fp = os.path.join(os.getcwd(),'training_set','aux_data','sb_coast_polygon','sb_coast_polygon.shp')
coast = gpd.read_file(fp)
coast

,id,geometry
0,0,"POLYGON ((-120.54335 34.96932, -120.62329 34.9..."


In [4]:
coast_scenes = query_geom(coast.geometry[0],2018)
coast_scenes

[<Item id=ca_m_3412037_nw_10_060_20180913_20190208>,
 <Item id=ca_m_3412029_sw_10_060_20180913_20190209>,
 <Item id=ca_m_3412028_ne_10_060_20180913_20190209>,
 <Item id=ca_m_3412004_sw_10_060_20180913_20190208>,
 <Item id=ca_m_3412004_nw_10_060_20180913_20190208>,
 <Item id=ca_m_3412003_se_10_060_20180913_20190208>,
 <Item id=ca_m_3412003_ne_10_060_20180913_20190208>,
 <Item id=ca_m_3412039_nw_10_060_20180724_20190209>,
 <Item id=ca_m_3412038_nw_10_060_20180724_20190209>,
 <Item id=ca_m_3412038_ne_10_060_20180724_20190208>,
 <Item id=ca_m_3412037_ne_10_060_20180724_20190208>,
 <Item id=ca_m_3412012_nw_10_060_20180724_20190208>,
 <Item id=ca_m_3412004_ne_10_060_20180724_20190208>,
 <Item id=ca_m_3411945_nw_11_060_20180724_20190209>,
 <Item id=ca_m_3411945_ne_11_060_20180724_20190209>,
 <Item id=ca_m_3411937_sw_11_060_20180724_20190208>,
 <Item id=ca_m_3411937_se_11_060_20180724_20190208>,
 <Item id=ca_m_3411936_sw_11_060_20180724_20190209>,
 <Item id=ca_m_3411936_se_11_060_20180724_2019

In [5]:
def make_bbox(item):
    c = item.properties['proj:bbox']
    return box(c[0],c[1],c[2],c[3])

In [7]:
# make dataframe with collected scenes and relevant info
df = pd.DataFrame(coast_scenes, columns = ['item'])
df['date']=df.item.apply(lambda item : item.properties['datetime'])
df['crs']=df.item.apply(lambda item : item.properties['proj:epsg'])
df['bbox']=df.item.apply(make_bbox)
df.shape

(30, 4)

In [8]:
def group_flight(df,date):
    same_date = df[df['date']==date]
    
    crs_list = same_date.crs.unique()
    polygons = []
    for crs in crs_list:
        same_crs = same_date[same_date['crs']==crs]
        area = shapely.ops.unary_union(same_crs.bbox)
        gdf = gpd.GeoDataFrame({'geometry':[area]}, 
                               crs='EPSG:'+str(crs))
        gdf.to_crs('EPSG:4326',inplace=True)
        polygons.append(gdf.geometry[0])

        flight = shapely.ops.unary_union(polygons)

    return flight

def flight_paths(df):
    dates = df.date.unique()
    flights = []
    for date in dates:
        flights.append(group_flight(df,date))
    gdf = gpd.GeoDataFrame({'date':dates, 'geometry':flights},
                     crs = 'EPSG:4326')
    return gdf

In [9]:
flights = flight_paths(df)

In [10]:
flights

,date,geometry
0,2018-09-13T00:00:00Z,"MULTIPOLYGON (((-120.50354 34.43618, -120.5016..."
1,2018-07-24T00:00:00Z,"MULTIPOLYGON (((-119.50156 34.30983, -119.5034..."
2,2018-07-22T00:00:00Z,"MULTIPOLYGON (((-119.81405 34.37217, -119.8140..."
3,2018-07-15T00:00:00Z,"POLYGON ((-119.99846 34.43456, -119.99607 34.5..."


In [11]:
flights.to_file(os.path.join(os.getcwd(),'training_set','aux_data','flights_coast.shp'))

/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
